In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

# GPU available?
device = torch.device("cuda" if torch.cuda.is_available() else "CPU")
print(f'using device: {device}')

using device: cuda


In [ ]:
train_dir = r"C:\Users\user\Documents\DLab\cifar10_train"
test_dir = r"C:\Users\user\Documents\DLab\cifar10_test"

# data normalization and augmentation
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.ImageFolder(train_dir, transform = transform)
test_dataset = datasets.ImageFolder(test_dir, transform = transform)

# create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size = 64,
    shuffle = True
)

test_loader = DataLoader(
    test_dataset,
    batch_size = 64,
    shuffle = False
)

In [ ]:
# vanilla CNN

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size= 3, stride = 1, padding = 1) #conv layer 1
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0) # pool
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1) # conv layer 2
        self.fc1 = nn.Linear(64 * 8 * 8, 128) # fully connected layer
        self.fc2 = nn.Linear(128, 256) # fully connected layer -- another
        self.fc3 = nn.Linear(256, 10) # output layer - 10 classes

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x))) # Conv1 --> ReLU --> Pool
        x = self.pool(nn.functional.relu(self.conv2(x))) # Conv2 --> ReLU --> Pool
        x = x.view(-1, 64 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x) # Output layer
        return x

In [ ]:
# initialising the model, loss function, optimiser

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# training the model

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:

        # move data to GPU
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{num_epochs}], loss: {running_loss/len(train_loader): .4f}")


In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100 * correct / total
        val_acc = evaluate(model, val_loader)
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')


In [ ]:
# evaluating on the test set

model.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:

        # move data to GPU
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct/total
print(f"Test accuracy: {test_accuracy: .2f}%")